In [2]:
import hashlib
import json
from time import time
import os

class Block:
    def __init__(self, index, carbon_data, timestamp, previous_hash):
        self.index = index
        self.carbon_data = carbon_data  # 存储碳数据
        self.timestamp = timestamp
        self.previous_hash = previous_hash
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        block_string = json.dumps(self.__dict__, sort_keys=True)
        return hashlib.sha256(block_string.encode()).hexdigest()

class CarbonBlockchain:
    def __init__(self):
        self.chain = []
        self.create_genesis_block()

    def create_genesis_block(self):
        genesis_block = Block(0, [], time(), "0")
        self.chain.append(genesis_block)

    def add_carbon_data(self, user_id, carbon_amount):
        previous_block = self.chain[-1]
        carbon_data = {'产品': user_id, 'carbon_amount': carbon_amount}
        new_block = Block(len(self.chain), carbon_data, time(), previous_block.hash)
        self.chain.append(new_block)

    def is_chain_valid(self):
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            previous_block = self.chain[i - 1]

            if current_block.hash != current_block.calculate_hash():
                return False

            if current_block.previous_hash != previous_block.hash:
                return False

        return True
    def load_from_data(self, chain_data):
        # 首先清除现有的链（如果需要）
        self.chain = []

        # 对于链数据中的每个区块数据
        for block_data in chain_data:
            # 创建一个新的Block实例
            block = Block(
                index=block_data['index'],
                carbon_data=block_data['carbon_data'],
                timestamp=block_data['timestamp'],
                previous_hash=block_data['previous_hash']
            )
            # 将新创建的区块添加到链上
            self.chain.append(block)

In [ ]:
# 示例使用
blockchain = CarbonBlockchain()

import sys
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QLabel, QLineEdit, QPushButton, QMessageBox
from PyQt5.QtGui import QFont, QPixmap, QPalette, QBrush
from PyQt5.QtCore import Qt


class RegisterWindow(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        self.setWindowTitle('注册')
        self.setGeometry(100, 100, 300, 120)

        layout = QVBoxLayout(self)

        self.username_entry = QLineEdit(self)
        self.username_entry.setPlaceholderText('请输入用户名')
        layout.addWidget(self.username_entry)

        self.password_entry = QLineEdit(self)
        self.password_entry.setPlaceholderText('密码')
        self.password_entry.setEchoMode(QLineEdit.Password)
        layout.addWidget(self.password_entry)

        register_button = QPushButton('注册', self)
        register_button.clicked.connect(self.onRegister)
        layout.addWidget(register_button)
    def save_user_data(self, username, password):  # 在这里添加 self
        user_data = {}
        user_data_file = 'user_data.json'

        # 如果文件已经存在，先读取现有数据
        if os.path.exists(user_data_file):
            with open(user_data_file, 'r') as file:
                user_data = json.load(file)

        # 将新用户的数据添加到字典中
        user_data[username] = password  # 注意：实际应用中应使用密码哈希

        # 将更新后的用户数据写回文件
        with open(user_data_file, 'w') as file:
            json.dump(user_data, file)

    def onRegister(self):
        username = self.username_entry.text().strip()
        password = self.password_entry.text()

        if not username or not password:
            QMessageBox.warning(self, '错误', '请输入用户名和密码')
            return

        self.save_user_data(username, password)  # 调用之前创建的保存用户数据的函数
        QMessageBox.information(self, '注册成功', '用户注册成功')
        self.close()


class LoginWindow(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        self.setWindowTitle('登录')
        self.setGeometry(100, 100, 300, 120)
        
  # 创建布局
        layout = QVBoxLayout(self)  # 此行确保布局实例被正确创建
    
         # 接下来的部件都将添加到这个布局中
        self.username_entry = QLineEdit(self)
        self.username_entry.setPlaceholderText('请输入用户名')
        layout.addWidget(self.username_entry)

        self.password_entry = QLineEdit(self)
        self.password_entry.setPlaceholderText('密码')
        self.password_entry.setEchoMode(QLineEdit.Password)
        layout.addWidget(self.password_entry)

        login_button = QPushButton('登录', self)
        login_button.clicked.connect(self.onLogin)
        layout.addWidget(login_button)
        
        register_button = QPushButton('注册新用户', self)
        register_button.clicked.connect(self.openRegisterWindow)
        layout.addWidget(register_button)
        # 设置主布局
        self.setLayout(layout)
    def openRegisterWindow(self):
        self.register_window = RegisterWindow()
        self.register_window.show()
 
    def validate_user(self,username, password):
        user_data_file = 'user_data.json'
        if os.path.exists(user_data_file):
            with open(user_data_file, 'r') as file:
                user_data = json.load(file)
                stored_password = user_data.get(username)
                return stored_password == password
        return False    

    def onLogin(self):
        username = self.username_entry.text().strip()
        password = self.password_entry.text()

        if username and password:
            if self.validate_user(username, password):  # 调用验证函数
                self.open_main_window(username)
            else:
                QMessageBox.warning(self, '错误', '用户名或密码不正确')
        else:
            QMessageBox.warning(self, '错误', '请输入用户名和密码')

    def open_main_window(self, username):
        # 这里打开主窗口
        self.main_window = CarbonFootprintApp(username)
        self.main_window.show()
        self.close()
        

class CarbonFootprintApp(QWidget):
     
    def __init__(self, username):
        super().__init__()
        self.username = username
        self.initUI()
        self.load_data()
    def initUI(self):
        # 设置窗口基本属性
        self.setWindowTitle('碳标签录入器')
        self.setGeometry(100, 100, 400, 300)
        self.setStyleSheet("background-color: white;")
        

        # 创建垂直布局
        layout = QVBoxLayout()

        # 添加标题
        title = QLabel('碳标签录入器')
        title_font = QFont('Arial', 20, QFont.Bold)
        title.setFont(title_font)
        title.setStyleSheet("color: green;")
        layout.addWidget(title, alignment=Qt.AlignCenter)

        # 创建产品信息输入框
        self.product_info_entry = self.create_line_edit('输入产品信息（条形码/名称）')
        layout.addWidget(self.product_info_entry)

        # 创建碳标签数据输入框
        self.carbon_label_entry = self.create_line_edit('输入碳标签数据')
        layout.addWidget(self.carbon_label_entry)

        # 创建提交按钮
        submit_button = QPushButton('提交', self)
        submit_button.clicked.connect(self.onSubmit)
        submit_button.setStyleSheet("QPushButton { background-color: green; color: white; border-radius: 10px; }")
        layout.addWidget(submit_button, alignment=Qt.AlignCenter)

        # 创建“显示区块链数据”按钮
        show_data_button = QPushButton('显示区块链数据', self)
        show_data_button.clicked.connect(self.onShowData)
        layout.addWidget(show_data_button, alignment=Qt.AlignCenter)
        # 设置主布局
        self.setLayout(layout)

    def load_data(self):
        try:
            with open(f'{self.username}_data.json', 'r') as file:
                data = json.load(file)
                # 确保区块链类有相应的方法来处理加载的数据
                blockchain.load_from_data(data['chain'])
        except FileNotFoundError:
            pass  # 文件不存在，意味着是新用户或之前没有保存过数据
    def create_line_edit(self, placeholder):
        line_edit = QLineEdit(self)
        line_edit.setPlaceholderText(placeholder)
        line_edit.setStyleSheet("QLineEdit { border: 2px solid green; border-radius: 10px; padding: 5px; }")
        return line_edit

    def onSubmit(self):
        product_info = self.product_info_entry.text()
        carbon_label = self.carbon_label_entry.text()

        # 检查输入有效性
        if not product_info.strip() or not carbon_label.strip():
            QMessageBox.warning(self, '错误', '请输入产品信息和碳标签数据')
            return

        try:
            carbon_amount = int(carbon_label)
        except ValueError:
            QMessageBox.warning(self, '错误', '碳标签数据应为数字')
            return

        # 将数据添加到区块链
        blockchain.add_carbon_data(product_info, carbon_amount)
        QMessageBox.information(self, '提交成功', f'产品信息: {product_info}\n碳标签: {carbon_label}')
    def onShowData(self):
        blockchain_data = ""
        for i, block in enumerate(blockchain.chain):
            if i > 0:  # 跳过创世区块
                blockchain_data += f"区块 {block.index}: {block.carbon_data}\n"

        QMessageBox.information(self, '区块链数据', blockchain_data)
    def verify_chain(self):
        # 验证创世区块
        if self.chain[0].hash != self.chain[0].calculate_hash():
            return False, 0

        # 验证余下的区块
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            previous_block = self.chain[i - 1]

            # 重新计算当前区块的哈希值并进行检查
            if current_block.hash != current_block.calculate_hash():
                return False, i

            # 检查当前区块的previous_hash是否等于上一个区块的哈希
            if current_block.previous_hash != previous_block.hash:
                return False, i

        return True, -1  # 如果所有区块都有效，则返回True
    def closeEvent(self, event):
        # 当窗口关闭时，保存数据
        data = {
            'chain': [vars(block) for block in blockchain.chain]
        }
        with open(f'{self.username}_data.json', 'w') as file:
            json.dump(data, file)
        event.accept()
if __name__ == '__main__':
    app = QApplication(sys.argv)
    login_window = LoginWindow()
    login_window.show()
    sys.exit(app.exec_())

In [3]:
filename = '110_data.json'
try:
    with open(filename, 'r') as file:
        data = json.load(file)
        # 现在 data 包含了文件中的数据
except FileNotFoundError:
    print(f"文件 {filename} 不存在。")
except json.JSONDecodeError:
    print(f"文件 {filename} 不包含有效的 JSON 数据。")

In [4]:
 data

{'chain': [{'index': 0,
   'carbon_data': [],
   'timestamp': 1701846943.226469,
   'previous_hash': '0',
   'hash': 'd9d8d3695eefb03d0f03cec52b08bb745f7267c78a4ea62463c637e4813a385b'},
  {'index': 1,
   'carbon_data': {'产品': '省赛', 'carbon_amount': 123},
   'timestamp': 1701846958.190943,
   'previous_hash': 'd9d8d3695eefb03d0f03cec52b08bb745f7267c78a4ea62463c637e4813a385b',
   'hash': 'ad5e7bbf2ef13386a68c1cd82b183c41bcb1c3e72c640d89cd3a6308dbc73c78'}]}